In [0]:
    


date='2019-12-11'
raw_df = spark.read.option("delimiter", "\t").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_app/*/".format(date)).show()


In [0]:

%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select feed_id,store_id,sum(estimate) from plproxy.execute_select_nestloop(\$proxy\$ 
    select feed_id,store_id,sum(estimate)
    from aa.app_store_daily_estimate_1
    where 
        date = '2019-09-01' 
    group by store_id,feed_id
\$proxy\$) tbl (feed_id SMALLINT,store_id INT, estimate INT) group by store_id,feed_id;

EOF


In [0]:

from pyspark.sql import types as T
import pandas as pd

EST_SCHEMA = T.StructType(
[
    T.StructField("country_code", T.StringType(), True),
    T.StructField("date", T.StringType(), True),
    T.StructField("device_code", T.StringType(), True),
    T.StructField("granularity", T.StringType(), True),
    T.StructField("free_app_download", T.StringType(), True),  
    T.StructField("app_id", T.StringType(), True),
    T.StructField("paid_app_download", T.StringType(), True),
    T.StructField("revenue", T.StringType(), True),
    T.StructField("revenue_iap", T.StringType(), True)    ]
)
    
def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

date_list = get_date_list("2018-03-07", "2019-07-13")
# date_list = get_date_list("2010-08-02", "2019-07-13")

for date in date_list:
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct()
    unifie_count = unified_df.count()
    # unified_df.write.parquet("s3://b2c-prod-data-pipeline-qa/tom/store/unified/date={}/".format(date))
    

    raw_df = spark.read.option("delimiter", "\t").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_app/*/".format(date)).select("_c5").distinct()
    raw_count= raw_df.count()
    # raw_df.write.parquet("s3://b2c-prod-data-pipeline-qa/tom/store/raw/date={}/".format(date))

    
    if unifie_count!=raw_count:
        print "{}; Fail; {};{}".format(date, raw_count, unifie_count)
    else:
        print "{}; Pass".format(date)